In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
    ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
    ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    #('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4447


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(10, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0042
Function value obtained: 0.4725
Current minimum: 0.4725
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0022
Function value obtained: 0.4733
Current minimum: 0.4725
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0021
Function value obtained: 0.4719
Current minimum: 0.4719
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0037
Function value obtained: 0.4732
Current minimum: 0.4719
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.7786
Function value obtained: 0.4728
Current minimum: 0.4717
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.7998
Function value obtained: 0.4738
Current minimum: 0.4717
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.7521
Function value obtained: 0.4718
Current minimum: 0.4717
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.7772
Function value obtained: 0.4731
Current minimum: 0.4717
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.7260
Function value obtained: 0.4737
Current minimum: 0.4717
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.0280
Function value obtained: 0.4726
Current minimum: 0.4716
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.0443
Function value obtained: 0.4734
Current minimum: 0.4716
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.1017
Function value obtained: 0.4739
Current minimum: 0.4716
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.0604
Function value obtained: 0.4732
Current minimum: 0.4716
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.1947
Function value obtained: 0.4717
Current minimum: 0.4716
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3251
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.3579
Function value obtained: 0.4382
Current minimum: 0.4371
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.4058
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.4296
Function value obtained: 0.4380
Current minimum: 0.4371
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.4772
Function value obtained: 0.4387
Current minimum: 0.4371
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.9004
Function value obtained: 0.4380
Current minimum: 0.4368
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.9447
Function value obtained: 0.4380
Current minimum: 0.4368
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.9328
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.8807
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.8917
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.2562
Function value obtained: 0.4650
Current minimum: 0.4650
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.3804
Function value obtained: 0.4656
Current minimum: 0.4650
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.4563
Function value obtained: 0.4637
Current minimum: 0.4637
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3789
Function value obtained: 0.4650
Current minimum: 0.4637
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3089
Function value obtained: 0.4653
Current minimum: 0.4637
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.1398
Function value obtained: 0.4660
Current minimum: 0.4636
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.8589
Function value obtained: 0.4664
Current minimum: 0.4636
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.8077
Function value obtained: 0.4636
Current minimum: 0.4636
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.8269
Function value obtained: 0.4662
Current minimum: 0.4636
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.8249
Function value obtained: 0.4657
Current minimum: 0.4636
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 1.3002
Function value obtained: 0.4635
Current minimum: 0.4635
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 1.2785
Function value obtained: 0.4669
Current minimum: 0.4635
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 1.2457
Function value obtained: 0.4635
Current minimum: 0.4635
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 1.2549
Function value obtained: 0.4661
Current minimum: 0.4635
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 1.4183
Function value obtained: 0.4634
Current minimum: 0.4634
Iteration No: 95 started. Sea

Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 0.4960
Function value obtained: 0.4381
Current minimum: 0.4377
Iteration No: 30 started. Searching for the next optimal point.
Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.5255
Function value obtained: 0.4382
Current minimum: 0.4377
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.5256
Function value obtained: 0.4385
Current minimum: 0.4377
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.8262
Function value obtained: 0.4378
Current minimum: 0.4377
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.7502
Function value obtained: 0.4384
Current minimum: 0.4377
Iteration No: 34 started. Sea

Iteration No: 69 ended. Search finished for the next optimal point.
Time taken: 1.0027
Function value obtained: 0.4376
Current minimum: 0.4376
Iteration No: 70 started. Searching for the next optimal point.
Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 1.2927
Function value obtained: 0.4380
Current minimum: 0.4376
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 1.0261
Function value obtained: 0.4389
Current minimum: 0.4376
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 1.0931
Function value obtained: 0.4387
Current minimum: 0.4376
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 1.2096
Function value obtained: 0.4392
Current minimum: 0.4376
Iteration No: 74 started. Sea

In [10]:
best_weights

{'cohesion': array([0.16240573, 0.08282686, 0.01624057, 0.11976722, 0.01624057,
        0.16240573, 0.0784168 , 0.07015325, 0.12913753, 0.16240573]),
 'syntax': array([0.15225017, 0.02024249, 0.04478987, 0.19634784, 0.02024249,
        0.2024249 , 0.05304506, 0.14970968, 0.14070501, 0.02024249]),
 'vocabulary': array([0.24103641, 0.02721076, 0.02721076, 0.27210756, 0.08951406,
        0.08808241, 0.04703807, 0.07278205, 0.10780717, 0.02721076]),
 'phraseology': array([0.20208263, 0.02020826, 0.02020826, 0.20208263, 0.02020826,
        0.17526739, 0.02020826, 0.09516657, 0.17637372, 0.06819401]),
 'grammar': array([0.20891405, 0.03402119, 0.03402119, 0.34021194, 0.03402119,
        0.03402119, 0.08572859, 0.07334767, 0.12169177, 0.03402119]),
 'conventions': array([0.17648661, 0.01781882, 0.01781882, 0.13360242, 0.09537523,
        0.12887179, 0.05749159, 0.11134062, 0.17818818, 0.08300594])}

In [11]:
scores

{'cohesion': 0.4715940959620777,
 'syntax': 0.4367411253114253,
 'vocabulary': 0.40473404738698004,
 'phraseology': 0.44626334311948995,
 'grammar': 0.4634333375391672,
 'conventions': 0.43745531892503614}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4434


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.944951,0.912791,0.909646,0.823066,0.886211
syntax,0.944951,1.000000,0.936865,0.959389,0.915805,0.929130
vocabulary,0.912791,0.936865,1.000000,0.957731,0.873953,0.890858
phraseology,0.909646,0.959389,0.957731,1.000000,0.942246,0.880402
grammar,0.823066,0.915805,0.873953,0.942246,1.000000,0.854456
conventions,0.886211,0.929130,0.890858,0.880402,0.854456,1.000000
